In [12]:
# Importing required libraries
import pandas as pd
import requests
import panel as pn
from panel.interact import interact
pn.extension()
import hvplot.pandas

In [13]:
# Using request to the API and storing it
response = requests.get('https://data.cityofnewyork.us/resource/jb7j-dtam.json?$limit=1000')
response1 = requests.get('https://data.cityofnewyork.us/resource/jb7j-dtam.json?$limit=1000&$offset=1001')
# Convert the response to a pandas DataFrame
data = pd.DataFrame(response.json())
data1 = pd.DataFrame(response1.json())
data_main = data.append(data1, ignore_index=True)

In [14]:
#Minor data cleaning
df = data_main.copy()
df['leading_cause']=df['leading_cause'].str.split('(').str[0].str.strip() 
df['deaths'] = pd.to_numeric(df['deaths'],errors='coerce')

In [15]:
# putting the values of year from data frame
start_year= int(data_main['year'].min())
end_year = int(data_main['year'].max())

In [16]:
#slider widget
year_slider = pn.widgets.IntSlider(name='Year Slider ::', start = start_year ,end= end_year, step=1, value=2007)
#function to create bar diagram with year slider
def df_year(y):
    df_year =  df[df['year']== str(y)]
    bar_year = df_year[['leading_cause','deaths']]
    bar_year = bar_year.groupby(['leading_cause']).sum().sort_values(by='deaths', ascending=False).head(5)
    return bar_year.hvplot.bar(y = 'deaths',height=500,width=800,xlabel=f'Leading Cause - {y}', ylabel= 'Numbers of Deaths')
interact(df_year,y= year_slider)

Column
    [0] Column
        [0] IntSlider(end=2014, name='Year Slider ::', start=2007, value=2007)
    [1] Row
        [0] HoloViews(Bars, name='interactive01245')

In [17]:
#function to create df with function slider
def chart(chart_slider):
    df_year =  df[df['year']== str(chart_slider)]
    bar_year_sex = df_year[['leading_cause','sex','deaths']].groupby(['leading_cause','sex']).sum('deaths').sort_values(by=['deaths'], ascending=False).head(6)
    return bar_year_sex
interact(chart,chart_slider=year_slider)

Column
    [0] Column
        [0] IntSlider(end=2014, name='Year Slider ::', start=2007, value=2007)
    [1] Row
        [0] DataFrame(DataFrame, name='interactive01353')

In [18]:
#creating lists of all races and causes
race_cols = list(df['race_ethnicity'].unique())
cause_cols = list(df['leading_cause'].unique())

In [19]:
# function to select race and cause
def df_race_cause(select_race,select_cause):
    df_race = df[df['race_ethnicity']== str(select_race)] 
    df_race_cause=df_race[df_race['leading_cause']== str(select_cause)]
    df_race_cause = df_race_cause[['year','deaths']].groupby(['year']).sum()
    return df_race_cause.hvplot('year','deaths',height= 400, width= 600)

In [20]:
#function to select only race
def df_race(select_race):
        df_race = df[df['race_ethnicity']== str(select_race)]
        df_yearly = df_race[['year','deaths']].groupby('year').sum()
        return df_yearly.hvplot('year','deaths',height= 400, width= 600)

In [21]:
#creating a checkbox 
cause_checkbox = pn.widgets.Checkbox(name='Select Cause of Death')

@pn.depends(cause_checkbox)
#function to select and deselect interactive button
def cause_check(x):
    if x == True:
        return interact(df_race_cause, select_race = race_cols, select_cause = cause_cols)
    else:
        return interact(df_race, select_race = race_cols)
        
pn.Column(pn.Column(cause_checkbox),cause_check)

Column
    [0] Column
        [0] Checkbox(name='Select Cause of Death')
    [1] ParamFunction(function)

In [23]:
# Layout using Template
template = pn.template.FastListTemplate(
    title='CAUSES OF DEATH - NYC dashboard', 
    main=[pn.Row(pn.Column(cause_check),cause_checkbox),
          pn.Row(pn.Column(interact(df_year,y= year_slider)),
                 pn.Column(interact(chart,chart_slider=year_slider)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();